# Assignment 8: M5 Competition Demand Forecasting with DeepAR-Nicholas Petr

## Import required packages and load datasets

In [1]:
import os
import numpy as np
import pandas as pd
#import mxnet
import re
from datetime import datetime
from pandas.tseries.holiday import USFederalHolidayCalendar as Calendar

In [2]:
calendar = pd.read_csv("calendar.csv")
print("calendar Shape:", calendar.shape)

sales_train = pd.read_csv("sales_train_validation.csv")
print("sales_train Shape:", sales_train.shape)

sell_prices = pd.read_csv("sell_prices.csv")
print("sell_prices Shape:", sell_prices.shape)

calendar Shape: (1969, 13)
sales_train Shape: (30490, 1918)
sell_prices Shape: (6841121, 4)


## 1. Which state has the highest sales?

In [3]:
sales_train.head()

,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4


In [4]:
sales_train['total'] = sales_train.loc[:, 'd_1':'d_1913'].sum(axis=1)
sales_train['total']

0         600
1         493
2         288
3        3288
4        1849
         ... 
30485    1022
30486     720
30487    1712
30488     702
30489    1329
Name: total, Length: 30490, dtype: int64

### California has the highest sales

In [5]:
sales_train.groupby('state_id')['total'].sum().sort_values(ascending=False)

state_id
CA    28675547
TX    18899006
WI    18120856
Name: total, dtype: int64

## 2. Which department has the highest sales?

### FOODS_3 has the highest sales 

In [6]:
sales_train.groupby('dept_id')['total'].sum().sort_values(ascending=False)

dept_id
FOODS_3        32372076
HOUSEHOLD_1    11500526
FOODS_2         7629822
HOBBIES_1       5596460
FOODS_1         5088041
HOUSEHOLD_2     2980144
HOBBIES_2        528340
Name: total, dtype: int64

## 3. Which department has the highest number of products?

### FOODS_3 has the highest number of products

In [7]:
sales_train.groupby('dept_id')['item_id'].nunique().sort_values(ascending=False)

dept_id
FOODS_3        823
HOUSEHOLD_1    532
HOUSEHOLD_2    515
HOBBIES_1      416
FOODS_2        398
FOODS_1        216
HOBBIES_2      149
Name: item_id, dtype: int64

## 4. Which department has the highest mean price?

In [8]:
sell_prices.head()

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


In [9]:
sell_prices['dept_id'] = sell_prices['item_id'].str[:-4]

### HOBBIES_1 has the highest mean price

In [10]:
sell_prices.groupby("dept_id")['sell_price'].mean().sort_values(ascending=False)

dept_id
HOBBIES_1      6.225784
HOUSEHOLD_2    5.855558
HOUSEHOLD_1    5.057583
FOODS_2        4.084008
FOODS_1        3.374649
FOODS_3        2.842107
HOBBIES_2      2.686807
Name: sell_price, dtype: float64

## 5. Which is the best performing store?

### CA_3 is the best performing store

In [11]:
sales_train.groupby("store_id")["total"].sum().sort_values(ascending=False)

store_id
CA_3    11188180
CA_1     7698216
TX_2     7214384
WI_2     6544012
WI_3     6427782
TX_3     6089330
CA_2     5685475
TX_1     5595292
WI_1     5149062
CA_4     4103676
Name: total, dtype: int64

## 6. Which month had the highest sales?

In [12]:
date_list = pd.date_range(start="2011-01-29",end="2016-04-24")
len(date_list)

1913

In [13]:
daily_sales_list = list(sales_train.loc[:, 'd_1':'d_1913'].sum(axis=0))
len(daily_sales_list)

1913

In [14]:
daily_sales = pd.DataFrame({"date": date_list, 
                               "daily_sales": daily_sales_list})
daily_sales.head()

,date,daily_sales
0,2011-01-29,32631
1,2011-01-30,31749
2,2011-01-31,23783
3,2011-02-01,25412
4,2011-02-02,19146


In [15]:
daily_sales["year"] = daily_sales["date"].dt.year
daily_sales["month"] = daily_sales["date"].dt.month
daily_sales["day"] = daily_sales["date"].dt.day

daily_sales.head()

,date,daily_sales,year,month,day
0,2011-01-29,32631,2011,1,29
1,2011-01-30,31749,2011,1,30
2,2011-01-31,23783,2011,1,31
3,2011-02-01,25412,2011,2,1
4,2011-02-02,19146,2011,2,2


### The month of March 2016 had the highest sales

In [16]:
daily_sales.groupby(["year", "month"])["daily_sales"].sum().sort_values(ascending=False)

year  month
2016  3        1275746
2015  8        1254341
      10       1248437
      7        1232147
2016  1        1215286
                ...   
2011  6         753380
      4         737713
      2         726375
      5         719562
      1          88163
Name: daily_sales, Length: 64, dtype: int64

## 7. Which weekday do people prefer to grocery shopping in different states?

## NOTE: I am making the assumption that "Weekday" means day of the *work* week, and therefore I am removing Saturday and Sunday from contention

In [17]:
daily_sales['day_of_week'] = daily_sales['date'].dt.dayofweek
daily_sales['day_of_week_text'] = daily_sales['date'].dt.day_name()

daily_sales.head()

,date,daily_sales,year,month,day,day_of_week,day_of_week_text
0,2011-01-29,32631,2011,1,29,5,Saturday
1,2011-01-30,31749,2011,1,30,6,Sunday
2,2011-01-31,23783,2011,1,31,0,Monday
3,2011-02-01,25412,2011,2,1,1,Tuesday
4,2011-02-02,19146,2011,2,2,2,Wednesday


In [18]:
state_sales = sales_train.groupby('state_id').sum().loc[:, 'd_1':'d_1913'].T

state_sales

state_id,CA,TX,WI
d_1,14195,9438,8998
d_2,13805,9630,8314
d_3,10108,6778,6897
d_4,11047,7381,6984
d_5,9925,5912,3309
...,...,...,...
d_1909,15148,9600,10595
d_1910,14488,9602,10943
d_1911,17095,10615,12807
d_1912,21834,12266,14862


In [19]:
daily_sales['CA'] = state_sales["CA"].values
daily_sales['TX'] = state_sales["TX"].values
daily_sales['WI'] = state_sales["WI"].values

daily_sales.head()

,date,daily_sales,year,month,day,day_of_week,day_of_week_text,CA,TX,WI
0,2011-01-29,32631,2011,1,29,5,Saturday,14195,9438,8998
1,2011-01-30,31749,2011,1,30,6,Sunday,13805,9630,8314
2,2011-01-31,23783,2011,1,31,0,Monday,10108,6778,6897
3,2011-02-01,25412,2011,2,1,1,Tuesday,11047,7381,6984
4,2011-02-02,19146,2011,2,2,2,Wednesday,9925,5912,3309


In [20]:
daily_sales_weekday = daily_sales[(daily_sales.day_of_week != 5) & (daily_sales.day_of_week != 6)]
daily_sales_weekday

,date,daily_sales,year,month,day,day_of_week,day_of_week_text,CA,TX,WI
2,2011-01-31,23783,2011,1,31,0,Monday,10108,6778,6897
3,2011-02-01,25412,2011,2,1,1,Tuesday,11047,7381,6984
4,2011-02-02,19146,2011,2,2,2,Wednesday,9925,5912,3309
5,2011-02-03,29211,2011,2,3,3,Thursday,11322,9006,8883
6,2011-02-04,28010,2011,2,4,4,Friday,12251,6226,9533
...,...,...,...,...,...,...,...,...,...,...
1906,2016-04-18,38059,2016,4,18,0,Monday,17052,9868,11139
1907,2016-04-19,37570,2016,4,19,1,Tuesday,15784,10922,10864
1908,2016-04-20,35343,2016,4,20,2,Wednesday,15148,9600,10595
1909,2016-04-21,35033,2016,4,21,3,Thursday,14488,9602,10943


### People prefer to grocery shop on Friday in California 

### People prefer to grocery shop on Monday in Texas 

### People prefer to grocery shop on Friday in Wisconsin 

In [21]:
daily_sales_weekday.groupby("day_of_week_text").sum()

,daily_sales,year,month,day,day_of_week,CA,TX,WI
day_of_week_text,,,,,,,,
Friday,9343694,549608,1731,4273,1092,4006162,2627034,2710498
Monday,8968860,549604,1742,4306,0,3967465,2634239,2367156
Thursday,8245967,549607,1733,4305,819,3536236,2388387,2321344
Tuesday,8290677,549605,1741,4247,273,3614175,2385601,2290901
Wednesday,8192736,549606,1737,4275,546,3522441,2369281,2301014


## 8. Which holiday or event recorded the highest sales?

In [22]:
cal = Calendar()
holidays = cal.holidays(start=date_list.min(), end=date_list.max())

daily_sales['holiday'] = daily_sales['date'].isin(holidays)

In [23]:
daily_sales

,date,daily_sales,year,month,day,day_of_week,day_of_week_text,CA,TX,WI,holiday
0,2011-01-29,32631,2011,1,29,5,Saturday,14195,9438,8998,False
1,2011-01-30,31749,2011,1,30,6,Sunday,13805,9630,8314,False
2,2011-01-31,23783,2011,1,31,0,Monday,10108,6778,6897,False
3,2011-02-01,25412,2011,2,1,1,Tuesday,11047,7381,6984,False
4,2011-02-02,19146,2011,2,2,2,Wednesday,9925,5912,3309,False
...,...,...,...,...,...,...,...,...,...,...,...
1908,2016-04-20,35343,2016,4,20,2,Wednesday,15148,9600,10595,False
1909,2016-04-21,35033,2016,4,21,3,Thursday,14488,9602,10943,False
1910,2016-04-22,40517,2016,4,22,4,Friday,17095,10615,12807,False
1911,2016-04-23,48962,2016,4,23,5,Saturday,21834,12266,14862,False


### 2015-07-03, which would have been the Federal Holiday associated with the 4th of July in 2015, recorded the highest sales for any individual event 

In [24]:
holiday_sales = daily_sales[daily_sales["holiday"]].reset_index(drop=True)
holiday_sales.sort_values(by="daily_sales", ascending=False)

,date,daily_sales,year,month,day,day_of_week,day_of_week_text,CA,TX,WI,holiday
42,2015-07-03,54154,2015,7,3,4,Friday,23408,14085,16661,True
23,2013-09-02,47072,2013,9,2,0,Monday,21384,13152,12536,True
50,2016-02-15,46152,2016,2,15,0,Monday,19231,12115,14806,True
43,2015-09-07,45762,2015,9,7,0,Monday,21191,12774,11797,True
13,2012-09-03,44525,2012,9,3,0,Monday,19074,13484,11967,True
33,2014-09-01,43256,2014,9,1,0,Monday,19179,12686,11391,True
44,2015-10-12,42240,2015,10,12,0,Monday,18473,11422,12345,True
32,2014-07-04,38593,2014,7,4,4,Friday,16997,11912,9684,True
49,2016-01-18,38340,2016,1,18,0,Monday,18004,10685,9651,True
41,2015-05-25,37817,2015,5,25,0,Monday,17642,10552,9623,True


### Merge holiday_sales with calendar to get specific events and event types 

Remove month and year from daily_sales, as it is causing issues witht the calendar merge 

In [25]:
daily_sales_adjusted = daily_sales.drop('year', 1)

In [26]:
daily_sales_adjusted = daily_sales_adjusted.drop('month', 1)

In [27]:
calendar["date"] = calendar["date"].astype("datetime64")

In [28]:
cols = list(daily_sales_adjusted.columns) + ['event_name_1', 'event_name_2', 'event_type_1', 'event_type_2']

In [29]:
daily_sales_merged = pd.merge(daily_sales_adjusted, calendar, left_on="date", right_on="date")[cols]

### The Super Bowl outputs the most sales for general events 

In [30]:
daily_sales_merged.groupby("event_name_1")["daily_sales"].sum().sort_values(ascending=False)

event_name_1
SuperBowl              245544
Purim End              215742
LaborDay               210773
PresidentsDay          206678
ValentinesDay          200974
StPatricksDay          200758
Easter                 197588
LentStart              184362
LentWeek2              178482
Eid al-Fitr            176338
ColumbusDay            175824
VeteransDay            175594
Ramadan starts         173898
EidAlAdha              172520
Mother's day           171057
IndependenceDay        168554
NBAFinalsEnd           167821
Pesach End             167251
Chanukah End           167134
MartinLutherKingDay    166893
OrthodoxChristmas      166721
MemorialDay            163498
NBAFinalsStart         159372
OrthodoxEaster         153108
Halloween              150297
Father's day           146248
Cinco De Mayo          131294
NewYear                128842
Thanksgiving           105650
Christmas                  78
Name: daily_sales, dtype: int64

### Religious events are the event type with the most sales 

In [31]:
daily_sales_merged.groupby("event_type_1")["daily_sales"].sum().sort_values(ascending=False)

event_type_1
Religious    1755556
National     1502384
Cultural     1198216
Sporting      572737
Name: daily_sales, dtype: int64

### Event_name_2 and event_type_2 have much fewer associated sales 

In [32]:
daily_sales_merged.groupby("event_name_2")["daily_sales"].sum().sort_values(ascending=False)

event_name_2
Cinco De Mayo     43203
Father's day      42494
OrthodoxEaster    38934
Easter            24018
Name: daily_sales, dtype: int64

In [33]:
daily_sales_merged.groupby("event_type_2")["daily_sales"].sum().sort_values(ascending=False)

event_type_2
Cultural     109715
Religious     38934
Name: daily_sales, dtype: int64